## Load the data
## Clean the data
## Modelling

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
X,y = data.drop('target',axis=1),data['target']

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [6]:
import torch
import torch.nn as nn

In [7]:
import numpy as np

In [8]:
X_train = torch.from_numpy(np.array(X_train).astype(np.float32))
y_train = torch.from_numpy(np.array(y_train).astype(np.float32))
X_test = torch.from_numpy(np.array(X_test).astype(np.float32))
y_test = torch.from_numpy(np.array(y_test).astype(np.float32))

In [9]:
X_train.shape

torch.Size([227, 13])

In [10]:
X_test.shape

torch.Size([76, 13])

In [11]:
y_train.shape

torch.Size([227])

In [12]:
y_test.shape

torch.Size([76])

#### Modelling

In [13]:
import torch.nn.functional as F

In [14]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13,64)
        self.fc2 = nn.Linear(64,128)
        self.fc3 = nn.Linear(128,256)
        self.fc4 = nn.Linear(256,512)
        self.fc5 = nn.Linear(512,1024)
        self.fc6 = nn.Linear(1024,512)
        self.fc7 = nn.Linear(512,1)
    
    def forward(self,X):
        preds = self.fc1(X)
        preds = F.relu(preds)
        preds = self.fc2(preds)
        preds = F.relu(preds)
        preds = self.fc3(preds)
        preds = F.relu(preds)
        preds = self.fc4(preds)
        preds = F.relu(preds)
        preds = self.fc5(preds)
        preds = F.relu(preds)
        preds = self.fc6(preds)
        preds = F.relu(preds)
        preds = self.fc7(preds)
        return F.sigmoid(preds)

In [15]:
device = torch.device('cuda')

In [16]:
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

In [17]:
PROJECT_NAME = 'Heart-Disease-UCI'

In [18]:
def get_loss(criterion,X,y,model):
    model.eval()
    with torch.no_grad():
        preds = model(X.float().to(device))
        preds = preds.to(device)
        y = y.to(device)
        loss = criterion(preds,y)
    model.train()
    return loss.item()
def get_accuracy(X,y,model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i in range(len(X)):
            pred = model(X[i].float().to(device))
            pred.to(device)
            if round(int(pred[0])) == round(int(y[i])):
                correct += 1
            total += 1
        if correct == 0:
            correct += 1
    model.train()
    return round(correct/total,3)

In [19]:
import wandb

In [20]:
from tqdm import tqdm

In [136]:
# EPOCHS = 212
EPOCHS = 150

In [137]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

100%|██████████| 150/150 [00:14<00:00, 10.67it/s]


loss,0.37226
val_loss,0.52059
accuracy,0.471
val_accuracy,0.408
_runtime,21
_timestamp,1621447756
_step,149


loss,▇▇▆▆▇█▇▅▇▇▇▇▆▇▆▇▃▇▆▇▆▅▃▆▂▆█▂▇▃▂▅▂▅█▂▃▃▁▃
val_loss,▂▂▂▄▁▁▁▁▁▁▁▁▃▁▇▁▂▁▄▁▄▆█▃▂▁▇▁▃▂▁▁▅▄▇▅▇▄▅▅
accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇█
_timestamp,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇█
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


In [138]:
preds

tensor([9.9668e-01, 9.9938e-01, 9.9739e-01, 9.9998e-01, 9.9903e-01, 9.9960e-01,
        9.9996e-01, 9.9973e-01, 2.6602e-02, 9.9098e-01, 9.9990e-01, 9.9994e-01,
        9.5160e-01, 9.9970e-01, 9.9918e-01, 9.9880e-01, 9.9962e-01, 9.9762e-01,
        9.9998e-01, 9.9803e-01, 9.9720e-01, 9.9430e-01, 9.9978e-01, 9.9996e-01,
        9.9974e-01, 9.9996e-01, 9.9774e-01, 9.9996e-01, 9.9987e-01, 9.9946e-01,
        6.9243e-03, 9.9995e-01, 9.3821e-01, 9.9947e-01, 4.2673e-04, 9.9286e-01,
        9.8938e-01, 9.9994e-01, 1.1568e-04, 9.6151e-01, 9.9983e-01, 9.9982e-01,
        1.5249e-03, 6.8518e-01, 9.9795e-01, 9.9668e-01, 9.9547e-01, 8.2238e-01,
        9.9930e-01, 9.9995e-01, 1.6853e-03, 9.9998e-01, 9.5828e-01, 9.9989e-01,
        4.2189e-04, 9.9998e-01, 9.9415e-01, 9.9243e-01, 8.0771e-01, 9.9990e-01,
        9.9995e-01, 6.7366e-03, 4.8897e-03, 9.9981e-01, 9.9991e-01, 9.9971e-01,
        9.9996e-01, 9.9997e-01, 1.2036e-02, 1.0629e-02, 9.9993e-01, 1.8467e-03,
        9.9703e-01, 1.0000e+00, 9.9106e-

In [139]:
preds = torch.round(preds)

In [140]:
correct = 0
total = 0

In [141]:
for real,pred in zip(y_train,preds):
    if real == pred:
        correct += 1
    total += 1

In [142]:
round(correct/total,3)

0.626